In [3]:
import csv
import sklearn
import pickle
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedKFold,cross_val_score,learning_curve

In [4]:
df = pd.read_excel('sms_data.xlsx' )
df.head()

,sms,class,Unnamed: 2,Unnamed: 3
0,*سمنانِ مَن*پرداخت درون شهری –تاکسی شهری 14 ت ...,ham,NaN,NaN
1,کد تایید : 4657شهرداری سمنان,ham,NaN,NaN
2,گوشت گوسفندی تازه 83500برنج ایرانی 31999همبرگر...,spam,NaN,NaN
3,سرمایه‌گذاری در پرسودترین صندوق بدون ریسک کشور...,spam,NaN,NaN
4,شقابق سلاماز شما دعوت می‌کنیم در وبینار پرفورم...,spam,NaN,NaN


In [6]:
train_data = df[:99]
test_data = df[100:400]
test_data

,sms,class,Unnamed: 2,Unnamed: 3
100,آواهای منتخب آخرين ماه پاييز: فعالسازی با ارس...,NaN,NaN,NaN
101,60 دقیقه مکالمه درون شبکه از ساعت 7 صبح تا 23،...,NaN,NaN,NaN
102,یه فوتباله و یه فوتبال برتر جدال سرخ آبی‌ها د...,NaN,NaN,NaN
103,تراکنش از شما، جایزه از مابا نصب و انجام تراکن...,NaN,NaN,NaN
104,جشنواره جوایز افق کوروش بهمراه تخفیفات شگفت ان...,NaN,NaN,NaN
...,...,...,...,...
395,سلام 2 میلیون واریز کردم,NaN,NaN,NaN
396,سلام ممنون .باشه دستت درد نکنه,NaN,NaN,NaN
397,سلام صبح بخیر ۱۰میتونی خیابون امام بیای؟,NaN,NaN,NaN
398,شارژ انجام شد.اعتبارشما 50001 ريال.اینترنت همر...,NaN,NaN,NaN


In [7]:
train_data['class'].value_counts(normalize=True)

spam    0.525253
ham     0.474747
Name: class, dtype: float64

In [8]:
test_data['class'].value_counts(normalize=True)

Series([], Name: class, dtype: float64)

In [9]:
train_data['sms'] =train_data['sms'].str.split()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
vocab = []
for sms in train_data['sms']:
  for word in sms:
    vocab.append(word)
vocab = list(set(vocab))
len(vocab)

1224

In [11]:
word_count = {unique_word: [0] * len(train_data['sms']) for unique_word in vocab}

for index, sms in enumerate(train_data['sms']):
  for word in sms:
    word_count[word][index] += 1
word_count = pd.DataFrame(word_count)

train_data_clean = pd.concat([train_data , word_count] , axis=1)
train_data_clean.head()

,sms,class,Unnamed: 2,Unnamed: 3,5گوشی,امشب,آنتن,اجاره,کارت,G-347469,دانشجو,باشید,باحجم,خودت,«چرخ,آنلاین,فعال‌سازی:http://i3l.ir/g4rFXو,دربازیک!یک,داغ‌ترین‌ها,همه,۲,باشماره,نماييد.https://mci.ir/prize1با,آشنايي,سالگی,اقامت,اینترنتی,بابا,زیر:http://yelloadwise.ir/Ggq4X0M,قابل,ندارد,پيگيري,صرفاً,نشست‌های,متخصصان,٥,ریسک,گیگ!💾,مبلغ,1400.06.20,...,بنزين,بده,جا,عودت=ارسال7,شويد.*1000#,PM]مشترک,صرفه,اختتامیه,نمونی!okcs.ir/18شرکت,code:,کمپینگ,آبادان,اينترنت,ذخیره,نمره,روزلینک:,لیگ,3750,«خودرو,من,نمائيد.,رسانه‌هاي,http://yelloadwise.ir/PaH7Bdd,چارخونه😍🎬دوبله,شد!,اسپورت,گیری#7777*دستگاه,تشویق,‌گذاری,برابر,دیجی‌‌کالا,هموطن,تجربه,2»,آزمون,عزيز,کشیدی,گوشت,خرید،از,اخبار
0,"[*سمنانِ, مَن*پرداخت, درون, شهری, –تاکسی, شهری...",ham,NaN,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[کد, تایید, :, 4657شهرداری, سمنان]",ham,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[گوشت, گوسفندی, تازه, 83500برنج, ایرانی, 31999...",spam,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,"[سرمایه‌گذاری, در, پرسودترین, صندوق, بدون, ریس...",spam,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[شقابق, سلاماز, شما, دعوت, می‌کنیم, در, وبینار...",spam,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
spam_message = train_data_clean[train_data_clean['class'] == "spam"]
ham_message = train_data_clean[train_data_clean['class'] == "spam"]